In [4]:
import os 
import sys
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from astropy.table import Table
from astroML.filters import savitzky_golay, wiener_filter
%matplotlib inline

In [5]:
## automatically reload any modules read below that might have changed (e.g. plots)
%load_ext autoreload
%autoreload 2
# importing ZI tools for thermal analysis: 
import sputnikTools as s 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
## specifications for SOC-i from Boone Tate based on CAD model
def getInputParamsSOCi(SOCi=""):   
    
    if (SOCi==""):
        SOCi = s.getInputParamsDefault()

    # area-weighted surface properties for the UW SOC-i satellite
    alphaSOCi, epsSOCi = getSOCiSurfaceParamsDefault()

    # surface emissivity
    SOCi['alphaS'] = alphaSOCi  
    SOCi['epsT'] = epsSOCi      
    SOCi['alphaIR'] = SOCi['epsT']  # usual ansatz
    
    # thermal inertia
    SOCi['mass'] = 2.6    # kg (from Boone Tate)
    SOCi['C'] = 768.0     # J/kg/K, aluminum
    # battery charging/power dissipation
    SOCi['etaCell'] = 0.2 # fraction of energy for charging
    
    # effective areas for absorption
    SOCi['etaS'] = 0.21 
    SOCi['etaE'] = 0.36
    
    ## orbit
    SOCi['h'] = 550.0     # in km, orbit's altitude
    SOCi['PorbMin'] = 96  # orbital period in minutes
    SOCi['etaP'] = 0.375# fraction of orbit in eclipse: 36/96 
    
    return SOCi

def getSOCiSurfaceParamsDefault(verbose=False):
    ### from Boone Tate
    # Sides(2): 64% Solar Panel, 19% Aluminum Panels(outside), 17% aluminum frame rails
    # Sides(2): 57% Solar Panel, 26% aluminum panel(outside), 17% aluminum frame rails
    # Top: 60% Solar panel, 40% Aluminum frame
    # Bottom: 38% aluminum frame, 62% PCB
    ## from provided tables
    alphaSolPan = 0.92
    epsSolPan = 0.85 
    CSolPan = 324
    
    alphaAlPanOut = 0.87
    epsAlPanOut = 0.81 
    CAlPanOut = 801

    alphaAlFrameR = 0.86 
    epsAlFrameR = 0.86 
    CAlFrameR = 768

    alphaAlFrame = 0.08 
    epsAlFrame = 0.15 
    CAlFrame = 768

    alphaPCB = 0.81 
    epsPCB = 0.90 
    CPCB = 1544

    # total area in square meters 
    SolPan =  
    AlPanOut = 4 * 0.19 + 4 * 0.26
    AlFrameR = 4 * 0.17 + 4 * 0.17 
    AlFrame = 1 * 0.4 + 1 * 0.38
    PCB = 1 * 0.62
    Atot = SolPan + AlPanOut + AlFrameR + AlFrame + PCB
    alphaSOCi = alphaSolPan*SolPan + alphaAlPanOut*AlPanOut + alphaAlFrameR*AlFrameR
    alphaSOCi = (alphaSOCi + alphaAlFrame*AlFrame + alphaPCB*PCB)/Atot
    epsSOCi = epsSolPan*SolPan + epsAlPanOut*AlPanOut + epsAlFrameR*AlFrameR
    epsSOCi = (epsSOCi + epsAlFrame*AlFrame + epsPCB*PCB)/Atot
    CSOCi = CSolPan*SolPan + CAlPanOut*AlPanOut + CAlFrameR*AlFrameR
    CSOCi = (CSOCi + CAlFrame*AlFrame + CPCB*PCB)/Atot
    if (verbose):
        print(SolPan, AlPanOut, AlFrameR, AlFrame, PCB)

    return alphaSOCi, epsSOCi 

In [7]:
# set default SOCi parameters 
SOCi = getInputParamsSOCi()
coldSOCi = s.getInputParamsCold(getInputParamsSOCi())  
hotSOCi = s.getInputParamsHot(getInputParamsSOCi())  
alphaSOCi = SOCi['alphaS']
epsSOCi = SOCi['epsT']

In [8]:
hotSOCi['Atot'] = 1.0
hotSOCi['etaS'] = 1.0
hotSOCi['etaE'] = 1.0
hotSOCi['etaCell'] = 0.0
Qsun2, Qref2, QIR2, Qdissip2 = s.getAllHeatQ(hotSOCi)
print(Qsun2, Qref2, QIR2, Qdissip2)
QinSun, QinEclipse = s.getQin(hotSOCi)
QinSun, QinEclipse

1180.9710000000002 21.01532155860408 174.6376194039525 0.0


(1376.6239409625568, 174.6376194039525)

In [9]:
coldSOCi['Atot'] = 1.0
coldSOCi['etaS'] = 1.0
coldSOCi['etaE'] = 1.0
coldSOCi['etaCell'] = 0.0
Qsun2, Qref2, QIR2, Qdissip2 = s.getAllHeatQ(coldSOCi)
print(Qsun2, Qref2, QIR2, Qdissip2)
QinSun, QinEclipse = s.getQin(coldSOCi)
QinSun, QinEclipse

1097.9210000000003 144.2049943263533 147.77029334180597 0.0


(1389.8962876681594, 147.77029334180597)

In [10]:
# set default SOCi parameters 
SOCi = getInputParamsSOCi()
coldSOCi = s.getInputParamsCold(getInputParamsSOCi())  
hotSOCi = s.getInputParamsHot(getInputParamsSOCi())  
alphaSOCi = SOCi['alphaS']
epsSOCi = SOCi['epsT']

In [12]:
## hot and cold cases for SOC-i (UW 2U CubeSat)
t1 = 'SOC-i hot case'  
t2 = 'SOC-i cold case'  

# default randomized orientations
hotSOCi['etaS'] = 0.21 
hotSOCi['etaE'] = 0.36
coldSOCi['etaS'] = 0.21 
coldSOCi['etaE'] = 0.36

verbose = True
time, Temps1 = s.doOneCase(t1, alphaSOCi, epsSOCi, hotSOCi, verbose)  
time, Temps2 = s.doOneCase(t2, alphaSOCi, epsSOCi, coldSOCi, verbose)  

name = 'SOCi-hotVScold'
s.TempsPlot(time, Temps2, t2, Temps1, t1, "", "", name)


SOC-i hot case Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 24.80039100000001 0.7565515761097468 6.28695429854229 5.111388515221951
Qsun= 31.843896874652046  Qeclipse= 11.398342813764241
Hot Eq. Temp: 290.12K = 16.97℃
Cold Eq. Temp: 224.41K = -48.74℃
adjusting Tmax= 290.12800008941895 to: Thot= 290.1237666671349
doOneCase: temp. range = 289.4641556946817 289.83364290046774
     in Celsius: range = 16.314155694681745 16.68364290046776

SOC-i cold case Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 23.056341000000007 5.191379795748719 5.319730560305015 3.530965099468591
Qsun= 31.44887229637259  Qeclipse= 8.850695659773606
Hot Eq. Temp: 289.22K = 16.07℃
Cold Eq. Temp: 210.65K = -62.50℃
doOneCase: temp. range = 258.98122604261715 274.44572157726554
     in Celsius: range = -14.168773957382825 1.2957215772655672


In [13]:
## hot and cold cases for SOC-i (UW 2U CubeSat)
t1 = 'SOC-i hot case, extreme A'  
t2 = 'SOC-i cold case, extreme A'  

hotSOCi['etaS'] = 0.31 
hotSOCi['etaE'] = 0.38
coldSOCi['etaS'] = 0.10
coldSOCi['etaE'] = 0.34

verbose = True
time, Temps1 = s.doOneCase(t1, alphaSOCi, epsSOCi, hotSOCi, verbose)  
time, Temps2 = s.doOneCase(t2, alphaSOCi, epsSOCi, coldSOCi, verbose)  

name = 'SOCi-hotVScoldExtremeA'
s.TempsPlot(time, Temps2, t2, Temps1, t1, "", "", name)


SOC-i hot case, extreme A Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 36.61010100000001 0.798582219226955 6.636229537350196 7.481736643845394
Qsun= 44.04491275657716  Qeclipse= 14.11796618119559
Hot Eq. Temp: 314.63K = 41.48℃
Cold Eq. Temp: 236.74K = -36.41℃
doOneCase: temp. range = 313.7734606454903 314.559526678514
     in Celsius: range = 40.6234606454903 41.409526678514

SOC-i cold case, extreme A Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 10.979210000000004 4.902969807096012 5.024189973621403 1.985272475887002
Qsun= 19.71520629518522  Qeclipse= 7.009462449508405
Hot Eq. Temp: 257.35K = -15.80℃
Cold Eq. Temp: 198.72K = -74.43℃
doOneCase: temp. range = 235.51581095675334 244.2541254447327
     in Celsius: range = -37.634189043246636 -28.895874555267284


In [14]:
## extra heat on
t1 = 'SOC-i cold case'  
coldSOCi['etaS'] = 0.21 
coldSOCi['etaE'] = 0.36
Tcritical = 273.15
Pcritical = 5.0 
label = 'Extra 5 W when T$<$273 K'
name = 'SOCi-cold-heated5'

# analytic
timeA, TempsA = s.doOneCase(t1, alphaSOCi, epsSOCi, coldSOCi, verbose)   
# numerical test 
timeN, TempsN = s.getNumSolution(coldSOCi, 1.01*TempsA[0], Tcritical, Pcritical)
# plot
s.TempsPlotCompare(timeA, TempsA, timeN, TempsN, t1, label, name) 
print('Active control range:', np.min(TempsN-273.15), np.max(TempsN-273.15))



## extra heat on
t2 = 'SOC-i cold case'  
coldSOCi['etaS'] = 0.21 
coldSOCi['etaE'] = 0.36
Tcritical = 273.15
Pcritical = 2.0 
label = 'Extra 2 W when T$<$273 K'
name = 'SOCi-cold-heated2'

# analytic
timeA, TempsA = s.doOneCase(t2, alphaSOCi, epsSOCi, coldSOCi, verbose)   
# numerical test 
timeN, TempsN = s.getNumSolution(coldSOCi, 1.01*TempsA[0], Tcritical, Pcritical)
# plot
s.TempsPlotCompare(timeA, TempsA, timeN, TempsN, t2, label, name) 
print('Active control range:', np.min(TempsN-273.15), np.max(TempsN-273.15))


## extra heat on
t3 = 'SOC-i cold case'  
coldSOCi['etaS'] = 0.21 
coldSOCi['etaE'] = 0.36
Tcritical = 273.15
Pcritical = 10.0 
label = 'Extra 10 W when T$<$273 K'
name = 'SOCi-cold-heated10'

# analytic
timeA, TempsA = s.doOneCase(t3, alphaSOCi, epsSOCi, coldSOCi, verbose)   
# numerical test 
timeN, TempsN = s.getNumSolution(coldSOCi, 1.01*TempsA[0], Tcritical, Pcritical)
# plot
s.TempsPlotCompare(timeA, TempsA, timeN, TempsN, t3, label, name) 
print('Active control range:', np.min(TempsN-273.15), np.max(TempsN-273.15))



t4 = 'SOC-i cold case, extreme A'  
coldSOCi['etaS'] = 0.10
coldSOCi['etaE'] = 0.34
Tcritical = 273.15
Pcritical = 5.0 
label = 'Extra 5 W when T$<$273 K'
name = 'SOCi-cold-extrA-heated5'

# analytic
timeA, TempsA = s.doOneCase(t4, alphaSOCi, epsSOCi, coldSOCi, verbose)   
# numerical test 
timeN, TempsN = s.getNumSolution(coldSOCi, 1.01*TempsA[0], Tcritical, Pcritical)
# plot
s.TempsPlotCompare(timeA, TempsA, timeN, TempsN, t4, label, name) 
print('Active control range:', np.min(TempsN-273.15), np.max(TempsN-273.15))


SOC-i cold case Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 23.056341000000007 5.191379795748719 5.319730560305015 3.530965099468591
Qsun= 31.44887229637259  Qeclipse= 8.850695659773606
Hot Eq. Temp: 289.22K = 16.07℃
Cold Eq. Temp: 210.65K = -62.50℃
doOneCase: temp. range = 258.98122604261715 274.44572157726554
     in Celsius: range = -14.168773957382825 1.2957215772655672
Active control range: -7.029643321464789 6.401408390095526

SOC-i cold case Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 23.056341000000007 5.191379795748719 5.319730560305015 3.530965099468591
Qsun= 31.44887229637259  Qeclipse= 8.850695659773606
Hot Eq. Temp: 289.22K = 16.07℃
Cold Eq. Temp: 210.65K = -62.50℃
doOneCase: temp. range = 258.98122604261715 274.44572157726554
     in Celsius: range = -14.168773957382825 1.2957215772655672
Active control range: -10.073203396790575 4.47740863232

In [15]:
coldSOCi

{'Fsun': 1322,
 'FIR': 220,
 'rhoE': 0.25,
 'h': 550.0,
 'PorbMin': 96,
 'etaP': 0.375,
 'falb': 0.62,
 'fE': 0.8473786384498566,
 'Atot': 0.1,
 'etaS': 0.1,
 'etaE': 0.34,
 'alphaS': 0.8305000000000001,
 'epsT': 0.7926600000000001,
 'alphaIR': 0.7926600000000001,
 'mass': 2.6,
 'C': 768.0,
 'etaCell': 0.2}

In [16]:
SOCi

{'Fsun': 1372,
 'FIR': 240,
 'rhoE': 0.3,
 'h': 550.0,
 'PorbMin': 96,
 'etaP': 0.375,
 'falb': 0.62,
 'fE': 0.8473786384498566,
 'Atot': 0.1,
 'etaS': 0.21,
 'etaE': 0.36,
 'alphaS': 0.8305000000000001,
 'epsT': 0.7926600000000001,
 'alphaIR': 0.7926600000000001,
 'mass': 2.6,
 'C': 768.0,
 'etaCell': 0.2}

In [17]:
hotSOCi

{'Fsun': 1422,
 'FIR': 260,
 'rhoE': 0.35,
 'h': 550.0,
 'PorbMin': 96,
 'etaP': 0.0,
 'falb': 0.06,
 'fE': 0.8473786384498566,
 'Atot': 0.1,
 'etaS': 0.31,
 'etaE': 0.38,
 'alphaS': 0.8305000000000001,
 'epsT': 0.7926600000000001,
 'alphaIR': 0.7926600000000001,
 'mass': 2.6,
 'C': 768.0,
 'etaCell': 0.2}